In this notebook, I will experiment with a simple backtesting strategy using the `NautilusTrader` library. The strategy will be tested on Bitcoin/USD 1-hour interval data. This workflow will demonstrate how to set up, implement, and evaluate a trading strategy using historical data.

In [151]:
import os
import sys
# Add src folder to Python path
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

## Load data

In [152]:
import polars as pl
from src.utils.utils import polars_to_pandas
# from src.config import config

In [153]:
df = pl.read_parquet("../data/BTCUSDT_1h_tmp_365days.parquet")
df.head()

open_time,open,high,low,close,volume,close_time,quote_asset_volume,trades,taker_base_vol,taker_quote_vol,ignore
datetime[ms],f64,f64,f64,f64,f64,datetime[ms],f64,i64,f64,f64,str
2024-07-29 15:00:00,68228.01,68304.0,67668.34,68104.0,2569.35164,2024-07-29 15:59:59.999,1.7466e8,140700,1207.6142,8.2108e7,"""0"""
2024-07-29 16:00:00,68103.99,68118.0,66800.0,66940.01,3974.77521,2024-07-29 16:59:59.999,2.6730e8,196389,1680.9994,1.1305e8,"""0"""
2024-07-29 17:00:00,66940.01,67177.52,66428.0,67015.08,3084.77284,2024-07-29 17:59:59.999,2.0632e8,170837,1412.07947,9.4456e7,"""0"""
2024-07-29 18:00:00,67015.09,67547.32,66940.0,67419.99,1330.02008,2024-07-29 18:59:59.999,8.9536e7,76770,714.41138,4.8095e7,"""0"""
2024-07-29 19:00:00,67419.99,67549.73,67242.99,67303.05,693.95142,2024-07-29 19:59:59.999,4.6779e7,42145,326.29489,2.1996e7,"""0"""


### POC design — Data ingest + catalog write

In [154]:
from __future__ import annotations
import polars as pl
from pathlib import Path
from nautilus_trader.model import BarType
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from decimal import Decimal

In [155]:
CATALOG_PATH = Path.cwd() / "catalog"
CATALOG_PATH.mkdir(parents=True, exist_ok=True)

# Create a catalog instance
catalog = ParquetDataCatalog(CATALOG_PATH)

In [156]:
BTCUSD = TestInstrumentProvider.btcusdt_binance()
print(BTCUSD)

CurrencyPair(id=BTCUSDT.BINANCE, raw_symbol=BTCUSDT, asset_class=CRYPTOCURRENCY, instrument_class=SPOT, quote_currency=USDT, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=6, size_increment=0.000001, multiplier=1, lot_size=None, margin_init=0, margin_maint=0, maker_fee=0.001, taker_fee=0.001, info=None)


OBLIGATORIO: Pandas df con timestamp como indice

In [157]:
df = df.with_columns(
        (pl.col("open_time") + pl.duration(hours=1)).alias("timestamp"),
    ).to_pandas()

# Change order of columns
df = df.reindex(columns=["timestamp", "open", "high", "low", "close", "volume"])
df = df.set_index("timestamp")
df.head()

,open,high,low,close,volume
timestamp,,,,,
2024-07-29 16:00:00,68228.01,68304.00,67668.34,68104.00,2569.35164
2024-07-29 17:00:00,68103.99,68118.00,66800.00,66940.01,3974.77521
2024-07-29 18:00:00,66940.01,67177.52,66428.00,67015.08,3084.77284
2024-07-29 19:00:00,67015.09,67547.32,66940.00,67419.99,1330.02008
2024-07-29 20:00:00,67419.99,67549.73,67242.99,67303.05,693.95142


In [158]:
EURUSD_SPOT_1MIN_BARTYPE = BarType.from_str(
    f"{BTCUSD.id}-1-HOUR-LAST-EXTERNAL"
)

In [159]:
wrangler = BarDataWrangler(bar_type=EURUSD_SPOT_1MIN_BARTYPE, instrument=BTCUSD)
BARS = wrangler.process(df)

In [160]:
catalog.write_data([BTCUSD])

In [161]:
catalog.write_data(BARS)

In [162]:
# Read and analyze data from the catalog
# - Retrieve all instrument definitions
all_instruments = catalog.instruments()
print(f"All instruments:\n{all_instruments}")

# - Get all available bars
all_bars = catalog.bars()
print(f"All bars count: {len(all_bars)}")

All instruments:
[CurrencyPair(id=BTCUSDT.BINANCE, raw_symbol=BTCUSDT, asset_class=CRYPTOCURRENCY, instrument_class=SPOT, quote_currency=USDT, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=6, size_increment=0.000001, multiplier=1, lot_size=None, margin_init=0, margin_maint=0, maker_fee=0.001, taker_fee=0.001, info=None)]
All bars count: 17520


## Simple bt

In [163]:
from nautilus_trader.config import BacktestEngineConfig, BacktestDataConfig, BacktestVenueConfig, BacktestRunConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.model import Bar
from nautilus_trader.model import Money, Venue
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.enums import AccountType
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.model.currencies import BTC
from nautilus_trader.model.currencies import USDT

Configure Data

In [164]:
data_config = BacktestDataConfig(
        catalog_path = str(CATALOG_PATH),
        data_cls=Bar,
        instrument_id=BTCUSD.id,
        bar_types=[EURUSD_SPOT_1MIN_BARTYPE],
    )
data_config

BacktestDataConfig(catalog_path='/workspaces/Crypto-Backtester-Duel/notebooks/catalog', data_cls=<class 'nautilus_trader.model.data.Bar'>, catalog_fs_protocol=None, catalog_fs_storage_options=None, instrument_id=InstrumentId('BTCUSDT.BINANCE'), start_time=None, end_time=None, filter_expr=None, client_id=None, metadata=None, bar_spec=None, instrument_ids=None, bar_types=[BarType(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL)])

Configure Venue

In [165]:
VENUE = Venue("BINANCE")

venue_config = BacktestVenueConfig(
        name=str(VENUE),
        oms_type="NETTING",
        account_type="CASH",  # Spot CASH account (not for perpetuals or futures)
        base_currency=None,  # Multi-currency account
        starting_balances=[Money(1_000_000.0, USDT), Money(1.0, BTC)],
    )
venue_config

BacktestVenueConfig(name='BINANCE', oms_type='NETTING', account_type='CASH', starting_balances=[Money(1000000.00000000, USDT), Money(1.00000000, BTC)], base_currency=None, default_leverage=1.0, leverages=None, book_type='L1_MBP', routing=False, frozen_account=False, reject_stop_orders=True, support_gtd_orders=True, support_contingent_orders=True, use_position_ids=True, use_random_ids=False, use_reduce_only=True, bar_execution=True, bar_adaptive_high_low_ordering=False, trade_execution=False, modules=None, fill_model=None, latency_model=None, fee_model=None)

Configure Strategy

In [166]:
strategies = [
    ImportableStrategyConfig(
        strategy_path="nautilus_trader.examples.strategies.ema_cross:EMACross",
        config_path="nautilus_trader.examples.strategies.ema_cross:EMACrossConfig",
        config={
            "instrument_id": BTCUSD.id,
            "bar_type": EURUSD_SPOT_1MIN_BARTYPE,
            "fast_ema_period": 30,
            "slow_ema_period": 100,
            "trade_size": Decimal(1_000.0),
        },
    ),
]
strategies

[ImportableStrategyConfig(strategy_path='nautilus_trader.examples.strategies.ema_cross:EMACross', config_path='nautilus_trader.examples.strategies.ema_cross:EMACrossConfig', config={'instrument_id': InstrumentId('BTCUSDT.BINANCE'), 'bar_type': BarType(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL), 'fast_ema_period': 30, 'slow_ema_period': 100, 'trade_size': Decimal('1000')})]

Configure Engine:

The BacktestDataConfig objects are integrated into the backtesting framework through BacktestRunConfig:
(https://nautilustrader.io/docs/latest/concepts/data/#integration-with-backtestrunconfig)

In [167]:
# # Configure backtest engine
# config = BacktestEngineConfig(trader_id=TraderId("BACKTESTER-001"))

# # Build the backtest engine
# engine = BacktestEngine(config=config)

In [168]:
run_config = BacktestRunConfig(
    engine=BacktestEngineConfig(strategies=strategies),
    venues=[venue_config],
    data=[data_config],
)
run_config

BacktestRunConfig(venues=[BacktestVenueConfig(name='BINANCE', oms_type='NETTING', account_type='CASH', starting_balances=[Money(1000000.00000000, USDT), Money(1.00000000, BTC)], base_currency=None, default_leverage=1.0, leverages=None, book_type='L1_MBP', routing=False, frozen_account=False, reject_stop_orders=True, support_gtd_orders=True, support_contingent_orders=True, use_position_ids=True, use_random_ids=False, use_reduce_only=True, bar_execution=True, bar_adaptive_high_low_ordering=False, trade_execution=False, modules=None, fill_model=None, latency_model=None, fee_model=None)], data=[BacktestDataConfig(catalog_path='/workspaces/Crypto-Backtester-Duel/notebooks/catalog', data_cls=<class 'nautilus_trader.model.data.Bar'>, catalog_fs_protocol=None, catalog_fs_storage_options=None, instrument_id=InstrumentId('BTCUSDT.BINANCE'), start_time=None, end_time=None, filter_expr=None, client_id=None, metadata=None, bar_spec=None, instrument_ids=None, bar_types=[BarType(BTCUSDT.BINANCE-1-HOU

In [169]:
node = BacktestNode(configs=[run_config])

results = node.run()

2025-07-30T15:34:17.899078961Z [INFO] BACKTESTER-001.BacktestEngine: Building system kernel
2025-07-30T15:34:17.899117246Z [INFO] BACKTESTER-001.MessageBus: config.database=None
2025-07-30T15:34:17.899119945Z [INFO] BACKTESTER-001.MessageBus: config.encoding='msgpack'
2025-07-30T15:34:17.899121753Z [INFO] BACKTESTER-001.MessageBus: config.timestamps_as_iso8601=False
2025-07-30T15:34:17.899122958Z [INFO] BACKTESTER-001.MessageBus: config.buffer_interval_ms=None
2025-07-30T15:34:17.899124554Z [INFO] BACKTESTER-001.MessageBus: config.autotrim_mins=None
2025-07-30T15:34:17.899126715Z [INFO] BACKTESTER-001.MessageBus: config.use_trader_prefix=True
2025-07-30T15:34:17.899128529Z [INFO] BACKTESTER-001.MessageBus: config.use_trader_id=True
2025-07-30T15:34:17.899129706Z [INFO] BACKTESTER-001.MessageBus: config.use_instance_id=False
2025-07-30T15:34:17.899131677Z [INFO] BACKTESTER-001.MessageBus: config.streams_prefix='stream'
2025-07-30T15:34:17.899136001Z [INFO] BACKTESTER-001.MessageBus: con

2024-07-29T16:00:00.000000000Z [ERROR] BACKTESTER-001.EMACross: Received <Bar[0]> data for unknown bar type
2024-08-02T19:00:00.000000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountBalanceNegative(balance=-255.62489500, currency=BTC)


2025-07-30T15:34:18.075085009Z [INFO] BACKTESTER-001.DataClient-BINANCE: DISPOSED
2025-07-30T15:34:18.075129696Z [INFO] BACKTESTER-001.DataEngine: DISPOSED
2025-07-30T15:34:18.075161822Z [INFO] BACKTESTER-001.RiskEngine: DISPOSED
2025-07-30T15:34:18.075200019Z [INFO] BACKTESTER-001.ExecClient-BINANCE: DISPOSED
2025-07-30T15:34:18.075223726Z [INFO] BACKTESTER-001.ExecEngine: DISPOSED
2025-07-30T15:34:18.075244457Z [INFO] BACKTESTER-001.MessageBus: Closed message bus
2025-07-30T15:34:18.075269692Z [INFO] BACKTESTER-001.BACKTESTER-001: Cleared actors
2025-07-30T15:34:18.075297787Z [INFO] BACKTESTER-001.EMACross: DISPOSED
2025-07-30T15:34:18.075320070Z [INFO] BACKTESTER-001.BACKTESTER-001: Cleared trading strategies
2025-07-30T15:34:18.075335565Z [INFO] BACKTESTER-001.BACKTESTER-001: Cleared execution algorithms
2025-07-30T15:34:18.077332203Z [INFO] BACKTESTER-001.BACKTESTER-001: DISPOSED


In [170]:
results

[BacktestResult(trader_id='BACKTESTER-001', machine_id='codespaces-1bbeae', run_config_id='1d0def9f9d8d27113f49f971304620b084621c590e8eaca46d783e7e00c2b8ce', instance_id='3f1849a0-ec80-41b2-93ef-6c39f5ff43e8', run_id='54bc1073-3ff8-4371-a6ec-a371daf51707', run_started=1753889657996162000, run_finished=1753889658070433000, backtest_start=1722268800000000000, backtest_end=1722625200000000000, elapsed_time=356400.0, iterations=199, total_events=3, total_orders=1, total_positions=0, stats_pnls={'USDT': {'PnL (total)': 0.0, 'PnL% (total)': 0.0, 'Max Winner': 0.0, 'Avg Winner': 0.0, 'Min Winner': 0.0, 'Min Loser': 0.0, 'Avg Loser': 0.0, 'Max Loser': 0.0, 'Expectancy': 0.0, 'Win Rate': 0.0}, 'BTC': {'PnL (total)': 0.0, 'PnL% (total)': 0.0, 'Max Winner': 0.0, 'Avg Winner': 0.0, 'Min Winner': 0.0, 'Min Loser': 0.0, 'Avg Loser': 0.0, 'Max Loser': 0.0, 'Expectancy': 0.0, 'Win Rate': 0.0}}, stats_returns={'Returns Volatility (252 days)': nan, 'Average (Return)': nan, 'Average Loss (Return)': nan, 